In [35]:
import pandas as pd

# Load the datasets
train_features = pd.read_csv(r"D:\Lomba-Lomba\Find IT UGM 2024\data-analytics-competition-find-it-2024\train_features.csv")
train_labels = pd.read_csv(r"D:\Lomba-Lomba\Find IT UGM 2024\data-analytics-competition-find-it-2024\train_labels.csv")
test_features = pd.read_csv(r"D:\Lomba-Lomba\Find IT UGM 2024\data-analytics-competition-find-it-2024\test_features.csv")
submission_format = pd.read_csv(r"D:\Lomba-Lomba\Find IT UGM 2024\data-analytics-competition-find-it-2024\submission_format.csv")


In [12]:
train_features.head()

,tahun_kelahiran,pendidikan,status_pernikahan,pendapatan,jumlah_anak_balita,jumlah_anak_remaja,terakhir_belanja,belanja_buah,belanja_daging,belanja_ikan,belanja_kue,pembelian_diskon,pembelian_web,pembelian_toko,keluhan,tanggal_menjadi_anggota
0,1979,Sarjana,Rencana Menikah,NaN,0.0,1.0,NaN,50575.0,260967.0,50575.0,20230.0,2.0,2.0,5.0,0.0,2014-05-05
1,1950,Sarjana,Rencana Menikah,84063000.0,NaN,NaN,70.0,6069.0,44506.0,80920.0,20230.0,9.0,6.0,4.0,0.0,2013-03-17
2,1966,Sarjana,Menikah,127532564.0,0.0,0.0,45.0,117611.0,265460.0,96341.0,145573.0,1.0,1.0,7.0,0.0,NaN
3,1961,Magister,Rencana Menikah,165579620.0,0.0,0.0,90.0,206346.0,1613901.0,27725.0,125868.0,0.0,7.0,8.0,0.0,NaN
4,1970,Sarjana,Rencana Menikah,117703159.0,1.0,1.0,78.0,90563.0,311757.0,40358.0,33875.0,7.0,6.0,5.0,0.0,NaN


In [ ]:
# Checking for missing values in both training and test datasets
missing_train = train_features.isnull().sum()
missing_train


In [ ]:
missing_test = test_features.isnull().sum()
missing_test

In [15]:
duplicate_train = train_features.duplicated().sum()
duplicate_test = test_features.duplicated().sum()

duplicate_train,
#duplicate_test


(0,)

In [36]:
# Membuat Variabel " umur_anggota " TRAIN
from datetime import datetime


tahun_saat_ini_train = datetime.now().year
train_features['usia'] = tahun_saat_ini_train - train_features['tahun_kelahiran']

# Membuat Variabel " umur_anggota " Test
from datetime import datetime


tahun_saat_ini_test = datetime.now().year
test_features['usia'] = tahun_saat_ini_test - train_features['tahun_kelahiran']

In [ ]:
#Jumlah anak

train_features['jumlah_anak'] = train_features['jumlah_anak_balita']+train_features['jumlah_anak_remaja']


### **HANDLING MISSING VALUE**

##### **imputasi biasa**

In [24]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Columns to be dropped
columns_to_drop = ['tanggal_menjadi_anggota', 'tahun_kelahiran']

# Categorical columns and numerical columns
categorical_columns = ['pendidikan', 'status_pernikahan']
numerical_columns = ['usia', 'pendapatan', 'jumlah_anak_balita', 'jumlah_anak_remaja', 
                     'terakhir_belanja', 'belanja_buah', 'belanja_daging', 'belanja_ikan', 
                     'belanja_kue', 'pembelian_diskon', 'pembelian_web', 'pembelian_toko', 'keluhan']

# Define imputers
categorical_imputer = SimpleImputer(strategy='most_frequent')
numerical_imputer = SimpleImputer(strategy='median')

# Define OneHotEncoder for categorical data
categorical_encoder = OneHotEncoder(handle_unknown='ignore')

# Define scaler for numerical data
scaler = StandardScaler()

# Preprocessing pipeline for categorical data
categorical_pipeline = Pipeline(steps=[
    ('imputer', categorical_imputer),
    ('encoder', categorical_encoder)
])

# Preprocessing pipeline for numerical data
numerical_pipeline = Pipeline(steps=[
    ('imputer', numerical_imputer),
    ('scaler', scaler)
])

# Combine categorical and numerical pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_pipeline, categorical_columns),
        ('num', numerical_pipeline, numerical_columns)
    ])

# Fit preprocessor on train features and transform both train and test features
train_features_preprocessed = preprocessor.fit_transform(train_features.drop(columns=columns_to_drop))
test_features_preprocessed = preprocessor.transform(test_features.drop(columns=columns_to_drop))

# Check the shape of the processed data
train_features_preprocessed.shape, test_features_preprocessed.shape


((3817, 25), (3818, 25))

##### **mice**

In [33]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Columns to be dropped
columns_to_drop = ['tanggal_menjadi_anggota', 'tahun_kelahiran']

# Categorical columns and numerical columns
categorical_columns = ['pendidikan', 'status_pernikahan']
numerical_columns = ['usia', 'pendapatan', 'jumlah_anak_balita', 'jumlah_anak_remaja', 
                     'terakhir_belanja', 'belanja_buah', 'belanja_daging', 'belanja_ikan', 
                     'belanja_kue', 'pembelian_diskon', 'pembelian_web', 'pembelian_toko', 'keluhan']

# Define imputers
categorical_imputer = SimpleImputer(strategy='most_frequent')
numerical_imputer = IterativeImputer()

# Define OneHotEncoder for categorical data
categorical_encoder = OneHotEncoder(handle_unknown='ignore')

# Define scaler for numerical data
scaler = StandardScaler()

# Preprocessing pipeline for categorical data
categorical_pipeline = Pipeline(steps=[
    ('imputer', categorical_imputer),
    ('encoder', categorical_encoder)
])

# Preprocessing pipeline for numerical data
numerical_pipeline = Pipeline(steps=[
    ('imputer', numerical_imputer),
    ('scaler', scaler)
])

# Combine categorical and numerical pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_pipeline, categorical_columns),
        ('num', numerical_pipeline, numerical_columns)
    ])

# Fit preprocessor on train features and transform both train and test features
train_features_preprocessed = preprocessor.fit_transform(train_features.drop(columns=columns_to_drop))
test_features_preprocessed = preprocessor.transform(test_features.drop(columns=columns_to_drop))

# Check the shape of the processed data
train_features_preprocessed.shape, test_features_preprocessed.shape


((3817, 25), (3818, 25))

##### **knn imputation**

In [37]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Columns to be dropped
columns_to_drop = ['tanggal_menjadi_anggota', 'pendidikan']

# Categorical columns and numerical columns
categorical_columns = ['usia', 'status_pernikahan']
numerical_columns = ['tahun_kelahiran', 'pendapatan', 'jumlah_anak_balita', 'jumlah_anak_remaja', 
                     'terakhir_belanja', 'belanja_buah', 'belanja_daging', 'belanja_ikan', 
                     'belanja_kue', 'pembelian_diskon', 'pembelian_web', 'pembelian_toko', 'keluhan']

# Define imputers
categorical_imputer = SimpleImputer(strategy='most_frequent')
numerical_imputer = KNNImputer()

# Define OneHotEncoder for categorical data
categorical_encoder = OneHotEncoder(handle_unknown='ignore')

# Define scaler for numerical data
scaler = StandardScaler()

# Preprocessing pipeline for categorical data
categorical_pipeline = Pipeline(steps=[
    ('imputer', categorical_imputer),
    ('encoder', categorical_encoder)
])

# Preprocessing pipeline for numerical data
numerical_pipeline = Pipeline(steps=[
    ('imputer', numerical_imputer),
    ('scaler', scaler)
])

# Combine categorical and numerical pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_pipeline, categorical_columns),
        ('num', numerical_pipeline, numerical_columns)
    ])

# Fit preprocessor on train features and transform both train and test features
train_features_preprocessed = preprocessor.fit_transform(train_features.drop(columns=columns_to_drop))
test_features_preprocessed = preprocessor.transform(test_features.drop(columns=columns_to_drop))

# Check the shape of the processed data
train_features_preprocessed.shape, test_features_preprocessed.shape


((3817, 95), (3818, 95))

In [6]:
len(train_features.columns)

16

In [7]:
train_features_preprocessed = pd.DataFrame(train_features_preprocessed)

In [ ]:
train_features_preprocessed

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
train_features_preprocessed.hist(figsize=(20,20))
plt.show()

### **Modelling**

In [38]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validation to evaluate the model
scores = cross_val_score(rf_classifier, train_features_preprocessed, train_labels.squeeze(), cv=60, scoring='f1_macro')

# Calculate average Macro F-Score across all folds
average_score = scores.mean()

average_score


0.7035611940049276

### **Tunning**

In [9]:
from sklearn.model_selection import GridSearchCV

# Definisikan hyperparameter yang akan dituning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, 
                           cv=5, scoring='f1_macro', verbose=2, n_jobs=-1)

# Lakukan pencarian parameter terbaik
grid_search.fit(train_features_preprocessed, train_labels.squeeze())

# Cetak parameter terbaik
print("Parameter terbaik:", grid_search.best_params_)

# Cetak nilai F1 score terbaik
print("Nilai F1 score terbaik:", grid_search.best_score_)


Fitting 5 folds for each of 162 candidates, totalling 810 fits


c:\Users\HAJRAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
405 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
316 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\HAJRAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\HAJRAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\HAJRAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\

Parameter terbaik: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Nilai F1 score terbaik: 0.7263224307081362


In [15]:
# Random Forest Tunning Hyperparameter

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(
                                       n_estimators=1050,
                                       max_depth= None,
                                       min_samples_split=2,
                                       min_samples_leaf = 1,
                                       random_state=42)

# Cross-validation to evaluate the model
scores = cross_val_score(rf_classifier, train_features_preprocessed, train_labels.squeeze(), cv=60, scoring='f1_macro')

# Calculate average Macro F-Score across all folds
average_score = scores.mean()

average_score


0.7460585162518013

In [23]:
# XGBOOST
from xgboost import XGBClassifier
import numpy as np

# Initialize the XGBoost Classifier
xgb_classifier = XGBClassifier(n_estimators=1000, random_state=42)

# Cross-validation to evaluate the model
xgb_scores = cross_val_score(xgb_classifier, train_features_preprocessed, train_labels.squeeze(), cv=60, scoring='f1_macro')

# Calculate average Macro F-Score across all folds
average_xgb_score = np.mean(xgb_scores)

average_xgb_score


0.7477576715801167

In [6]:
# SVM
from sklearn.svm import SVC
import numpy as np

# Initialize the SVM Classifier
svm_classifier = SVC(kernel='linear', random_state=42)

# Cross-validation to evaluate the model
svm_scores = cross_val_score(svm_classifier, train_features_preprocessed, train_labels.squeeze(), cv=60, scoring='f1_macro')

# Calculate average Macro F-Score across all folds
average_svm_score = np.mean(svm_scores)

average_svm_score


0.27017899796398553

In [6]:
# GRADIEN BOOSTING

from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

# Initialize the Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Cross-validation to evaluate the model
gb_scores = cross_val_score(gb_classifier, train_features_preprocessed, train_labels.squeeze(), cv=60, scoring='f1_macro')

# Calculate average Macro F-Score across all folds
average_gb_score = np.mean(gb_scores)

average_gb_score


0.5902689475998668

In [5]:
# NEURAL NETWORK
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
import numpy as np

# Initialize the MLP Classifier
nn_classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=10000, random_state=42)

# Cross-validation to evaluate the model
nn_scores = cross_val_score(nn_classifier, train_features_preprocessed, train_labels.squeeze(), cv=60, scoring='f1_macro')

# Calculate average Macro F-Score across all folds
average_nn_score = np.mean(nn_scores)

average_nn_score


0.5297460195522833

In [ ]:
# LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression
import numpy as np

# Initialize the Logistic Regression Classifier
logistic_classifier = LogisticRegression(random_state=42)

# Cross-validation to evaluate the model
logistic_scores = cross_val_score(logistic_classifier, train_features_preprocessed, train_labels.squeeze(), cv=60, scoring='f1_macro')

# Calculate average Macro F-Score across all folds
average_logistic_score = np.mean(logistic_scores)

average_logistic_score


In [10]:
# KNN
# from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
# Initialize the KNN Classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)

# Cross-validation to evaluate the model
knn_scores = cross_val_score(knn_classifier, train_features_preprocessed, train_labels.squeeze(), cv=60, scoring='f1_macro')

# Calculate average Macro F-Score across all folds
average_knn_score = np.mean(knn_scores)

average_knn_score


0.5724187033043248

In [9]:
# DDT

from sklearn.tree import DecisionTreeClassifier
import numpy as np

# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()

# Cross-validation to evaluate the model
dt_scores = cross_val_score(dt_classifier, train_features_preprocessed, train_labels.squeeze(), cv=60, scoring='f1_macro')

# Calculate average Macro F-Score across all folds
average_dt_score = np.mean(dt_scores)

average_dt_score


0.5754391312000043

In [11]:
# NAIVE BAYES

from sklearn.naive_bayes import GaussianNB
import numpy as np

# Initialize the Gaussian Naive Bayes Classifier
nb_classifier = GaussianNB()

# Cross-validation to evaluate the model
nb_scores = cross_val_score(nb_classifier, train_features_preprocessed, train_labels.squeeze(), cv=60, scoring='f1_macro')

# Calculate average Macro F-Score across all folds
average_nb_score = np.mean(nb_scores)

average_nb_score


0.13368360568249688

In [12]:
# Fit the XGBoost model to the entire training data
rf_classifier.fit(train_features_preprocessed, train_labels.squeeze(), )

# Predict on the test data
test_predictions = rf_classifier.predict(test_features_preprocessed)

# Create a DataFrame for submission
submission = pd.DataFrame({
    'ID': test_features['ID'],
    'jumlah_promosi': test_predictions
})

submission.head()


,ID,jumlah_promosi
0,2241,0
1,2274,0
2,1107,1
3,4478,0
4,5080,4


In [13]:
# Save the DataFrame to a CSV file
submission.to_csv(r'D:\Lomba-Lomba\Find IT UGM 2024\data-analytics-competition-find-it-2024\submission_predictions_11.csv', index=False)
